<a href="https://colab.research.google.com/github/DivyaDeswal/Projects/blob/main/Building_A_Simple_Chatbot_Using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT** **LIBRARIES**

In [10]:
import random
import numpy as np
import nltk
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem import WordNetLemmatizer

In [14]:
# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **DEFINE** **SAMPLE** **INTENTS**

In [15]:
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Howdy"],
            "responses": ["Hello!", "Hi there!", "Hey! How can I help you?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you", "Goodbye"],
            "responses": ["Goodbye!", "See you later!", "Have a nice day!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful"],
            "responses": ["You're welcome!", "No problem!", "Anytime!"]
        },
        {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Identify yourself"],
            "responses": ["I'm your chatbot assistant!", "Call me ChatBot.", "ChatBot at your service!"]
        }
    ]
}


# **PREPROCESSING**

## **Lemmatization** **& Tokenization**

In [16]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)  # Tokenize each sentence into words
        words.extend(tokens)  # Add to vocabulary
        documents.append((tokens, intent["tag"]))  # Pair tokenized words with their intent tag
        if intent["tag"] not in classes:
            classes.append(intent["tag"])  # Store unique class labels


tokenize: breaks sentences into words (e.g., "Hello there!" → ["Hello", "there"])

lemmatizer: reduces words to their base form (e.g., "running" → "run")



## **Clean** **Vocabulary**

In [17]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w.isalnum()]
words = sorted(set(words))  # Unique and sorted
classes = sorted(set(classes))  # Unique classes

Lowercases everything

Removes punctuation (isalnum() ensures only words/numbers stay)

Sorts and deduplicates the vocabulary and class list



# **PREPARE** **TRAINING** **DATA**

## **Convert** **Sentences** **to** **Bags** **of** **Words**

In [18]:
training_data = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in doc[0] if w.isalnum()]
    bag = [1 if w in pattern_words else 0 for w in words]  # Bag of words representation

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1  # One-hot encoding of label

    training_data.append([bag, output_row])

Each input sentence becomes a binary vector where:

1 = word is present in the sentence

0 = word is not

Each output is a one-hot vector matching the intent tag

## **Shuffle** **&** **Convert** **to** **NumPy** **Arrays**

In [19]:
random.shuffle(training_data)
training_data = np.array(training_data, dtype=object)
train_x = np.array(list(training_data[:, 0]))
train_y = np.array(list(training_data[:, 1]))

# **BUILD** **&** **TRAIN** **THE** **MODEL**

In [21]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(train_x, train_y, epochs=20, batch_size=5, verbose=1)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1769 - loss: 1.4069
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2538 - loss: 1.3601
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8096 - loss: 1.1011
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4558 - loss: 1.1577
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7596 - loss: 1.0903
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8231 - loss: 0.8570
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8865 - loss: 0.5595
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8865 - loss: 0.7035
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8231 - loss: 0.4764
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.2013
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9615 - loss: 0.2058
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.1271
E

# **Helper** **Functions**

In [22]:
def clean_input(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words if w.isalnum()]
    return sentence_words

In [23]:
# Bag of Words Representation
def bow(sentence, words):
    sentence_words = clean_input(sentence)
    return np.array([1 if w in sentence_words else 0 for w in words])

Creates a binary vector for the input sentence:

1 = word is present

0 = word is absent

This matches the vector format used during training.

In [31]:
# Predict Class (Intent)
def predict_class(sentence):
    input_data = bow(sentence, words).reshape(1, -1)
    probabilities = model.predict(input_data)[0]  ## Converts the input to BoW and reshapes for prediction.
                                                  ## Gets probabilities for each class.

    threshold = 0.2
    results = [[i, p] for i, p in enumerate(probabilities) if p > threshold]  # Keeps only predictions with a confidence > 0.2.
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]  # Returns the top intent(s) with their probabilities.


In [33]:
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        return "Sorry, I don't understand that."  # If no intent was confidently matched, respond with a fallback.
    tag = intents_list[0]['intent']
    for intent in intents_json['intents']:    # Finds the correct intent from the JSON.
                                              # Returns a random response from the matching intent.
         if intent['tag'] == tag:
            return random.choice(intent['responses'])

# **CHAT** **LOOP**

In [34]:
print("ChatBot is ready! Type 'quit' to exit.")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        print("ChatBot: Bye!")
        break
    intents_list = predict_class(message)
    response = get_response(intents_list, intents)
    print("ChatBot:", response)

ChatBot is ready! Type 'quit' to exit.
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
ChatBot: Hi there!
You: Goodbye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
ChatBot: See you later!
You: That's helpful
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
ChatBot: You're welcome!
You: What is your name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
ChatBot: Call me ChatBot.
You: Thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
ChatBot: No problem!
You: Howdy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
ChatBot: Hi there!
You: quit
ChatBot: Bye!
